In [2]:
#В домашней работе я разобрался, как работают базовые модели LinearRegression, LogisticRegression.

In [3]:
# Помимо этого сделал тесты логистической модели для нашего-датасета. Существовала проблема в датасете - дисбаланс классов
#~134000 True/ ~3000 False с помощью метода undersampling(реализовал тоже без сторонних библиотек). 

In [4]:
#ВАЖНО! В блокноте можно встретить импорт sklearn, но он тут исключительно в целях реализации кросс-валидации и никак не задействован в создании ТЗ.

In [5]:
# Начнем с классической модели линейной регрессии.

import numpy as np
import pandas as pd

class LinearRegression:
    def __init__(self, learning_rate=0.01, epochs=1000):
        """
        Инициализация параметров модели.

        learning_rate — скорость обучения,
        epochs — количество итераций.
        """
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.coefficients = None  # Коэффициенты модели (включая свободный член)

    def fit(self, X, y):
        """
        Обучение модели методом градиентного спуска.

        X — матрица признаков (numpy array или pandas DataFrame),
        y — целевая переменная (numpy array или pandas Series).
        """
        X = np.array(X)  # Преобразуем X в numpy array, если это DataFrame
        y = np.array(y)  # Преобразуем y в numpy array, если это Series

        # Добавляем столбец единиц для свободного члена
        X = np.c_[np.ones(X.shape[0]), X]

        # Инициализируем коэффициенты нулями
        self.coefficients = np.zeros(X.shape[1])

        # Количество примеров
        m = len(y)

        # Обучение с использованием градиентного спуска
        for epoch in range(self.epochs):
            predictions = X @ self.coefficients  # Предсказания текущей модели
            errors = predictions - y  # Ошибки предсказания
            gradient = (1 / m) * X.T @ errors  # Градиент
            self.coefficients -= self.learning_rate * gradient  # Обновляем коэффициенты

            # Для наблюдения: считаем текущий MSE и выводим каждые 100 эпох
            mse = np.mean(errors ** 2)
            if (epoch + 1) % 100 == 0 or epoch == 0:
                print(f"Эпоха {epoch + 1}, MSE: {mse:.4f}")

    def predict(self, X):
        """
        Предсказание на основе обученной модели.

        X — матрица признаков (numpy array или pandas DataFrame).
        """
        X = np.array(X)  # Преобразуем X в numpy array, если это DataFrame
        X = np.c_[np.ones(X.shape[0]), X]  # Добавляем столбец единиц
        return X @ self.coefficients  # Линейное уравнение


# Пример данных
X = np.array([[1, 2], [2, 3], [3, 4], [4, 5]])  # Матрица признаков
y = np.array([5, 7, 9, 11])  # Целевая переменная

# Создаем объект модели линейной регрессии
model = LinearRegression(learning_rate=0.01, epochs=1000)

# Обучаем модель
model.fit(X, y)

# Получаем предсказания
predictions = model.predict(X)
print("Предсказания:", predictions)


Эпоха 1, MSE: 69.0000
Эпоха 100, MSE: 0.1855
Эпоха 200, MSE: 0.1315
Эпоха 300, MSE: 0.0932
Эпоха 400, MSE: 0.0661
Эпоха 500, MSE: 0.0469
Эпоха 600, MSE: 0.0332
Эпоха 700, MSE: 0.0236
Эпоха 800, MSE: 0.0167
Эпоха 900, MSE: 0.0118
Эпоха 1000, MSE: 0.0084
Предсказания: [ 4.85405315  6.93135412  9.00865508 11.08595604]


In [6]:
# Так же продемонстрирую классическую реализацию логистической регрессии.

In [7]:
import numpy as np

class LogisticRegression:
    def __init__(self, learning_rate=0.01, num_iterations=1000):
        self.learning_rate = learning_rate
        self.num_iterations = num_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        """Сигмоида для преобразования значений."""
        return 1 / (1 + np.exp(-z))

    def initialize_parameters(self, n_features):
        """Инициализация параметров модели."""
        self.weights = np.zeros(n_features)
        self.bias = 0


    def fit(self, X, y):
        """Обучение модели с использованием градиентного спуска."""
        num_samples, num_features = X.shape
        self.initialize_parameters(num_features)

        for _ in range(self.num_iterations):
            # Прогнозы
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            # log-loss
            loss = -np.mean(y * np.log(y_predicted + 1e-15) + (1 - y) * np.log(1 - y_predicted + 1e-15))

            # Градиенты
            dw = (1 / num_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / num_samples) * np.sum(y_predicted - y)

            # Обновление параметров
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X, threshold=0.5):
        """Прогнозирование меток."""
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        return (y_predicted >= threshold).astype(int)

    def evaluate(self, y_true, y_pred):
        """Оценка метрик precision, recall, f1-score."""
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        fn = np.sum((y_true == 1) & (y_pred == 0))

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return {
            "precision": precision,
            "recall": recall,
            "f1_score": f1_score
        }

# Пример использования
if __name__ == "__main__":
    # Генерация данных
    np.random.seed(42)
    X = np.random.rand(100, 2)  # 100 образцов, 2 признака
    y = (np.sum(X, axis=1) > 1).astype(int)  # Простое разделение на классы

    # Разделение на тренировочные и тестовые данные
    split_idx = int(0.8 * len(X))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]

    # Создание и обучение модели
    model = LogisticRegression(learning_rate=0.1, num_iterations=1000)
    model.fit(X_train, y_train)

    # Прогнозы
    y_pred = model.predict(X_test)

    # Оценка метрик
    metrics = model.evaluate(y_test, y_pred)
    print("Precision:", metrics["precision"])
    print("Recall:", metrics["recall"])
    print("F1 Score:", metrics["f1_score"])


Precision: 1.0
Recall: 0.8888888888888888
F1 Score: 0.9411764705882353


In [4]:
'''
Перейдем к самой интересной части. Это реализация написана для нашего data-сета. Наша задача бинарной классификации, поэтому я решил использовать
модель логистической регрессии. Из проблемных моментов: в датасете к сожалению была найдена проблема дисбаланса классов. Задача решалась простым 
undersampling-ом. 
'''
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold


class CustomLogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        # Инициализация параметров модели
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def sigmoid(self, z):
        """
        Сигмоидная функция.
        """
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        """
        Обучение модели с использованием градиентного спуска.
        """
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)  # Инициализация весов
        self.bias = 0  # Инициализация смещения

        for _ in range(self.n_iterations):
            # Линейная комбинация
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self.sigmoid(linear_model)

            # Градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновление параметров
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        """
        Предсказание вероятностей и классификация.
        """
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self.sigmoid(linear_model)
        return np.array([1 if i > 0.5 else 0 for i in y_predicted])

    def evaluate(self, y_true, y_pred):
        """
        Оценка метрик: precision, recall, F1.
        """
        # Метрики
        tp = np.sum((y_true == 1) & (y_pred == 1))  # True positives
        tn = np.sum((y_true == 0) & (y_pred == 0))  # True negatives
        fp = np.sum((y_true == 0) & (y_pred == 1))  # False positives
        fn = np.sum((y_true == 1) & (y_pred == 0))  # False negatives

        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        return precision, recall, f1_score


# Загрузка данных
data = pd.read_csv('additional_variable_train.csv', delimiter=';', skipinitialspace=True)


# Выбор признаков и целевой переменной
X = data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]',
          'Torque [Nm]', 'Tool wear [min]', 'Sum_Parametr', 'Ratio']].values
y = data['Machine failure'].values

# Стандартизация данных
X = (X - X.mean(axis=0)) / X.std(axis=0)

# Балансировка классов методом undersampling
class_0_indices = np.where(y == 0)[0]
class_1_indices = np.where(y == 1)[0]
balanced_class_0_indices = np.random.choice(class_0_indices, size=len(class_1_indices), replace=False)
balanced_indices = np.concatenate([balanced_class_0_indices, class_1_indices])
np.random.shuffle(balanced_indices)

X_balanced = X[balanced_indices]
y_balanced = y[balanced_indices]

kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Метрики
metrics = {'precision': [], 'recall': [], 'f1_score': []}

# Создание модели
model = CustomLogisticRegression(learning_rate=0.01, n_iterations=10000)

# Кросс-валидация
for train_index, test_index in kf.split(X_balanced):
    X_train, X_test = X_balanced[train_index], X_balanced[test_index]
    y_train, y_test = y_balanced[train_index], y_balanced[test_index]

    # Обучение
    model.fit(X_train, y_train)

    # Предсказания
    predictions = model.predict(X_test)

    # Оценка метрик
    precision, recall, f1 = model.evaluate(y_test, predictions)

    metrics['precision'].append(precision)
    metrics['recall'].append(recall)
    metrics['f1_score'].append(f1)

    # Вывод метрик для каждого фолда
    print(f"Fold Results: Precision={precision:.2f}, Recall={recall:.2f}, F1={f1:.2f}")

# Итоговые метрики
print("\nAverage Metrics from Cross-Validation:")
print(f"Precision: {np.mean(metrics['precision']):.2f}")
print(f"Recall: {np.mean(metrics['recall']):.2f}")
print(f"F1 Score: {np.mean(metrics['f1_score']):.2f}")


Fold Results: Precision=0.98, Recall=0.80, F1=0.88
Fold Results: Precision=0.98, Recall=0.81, F1=0.88
Fold Results: Precision=0.97, Recall=0.80, F1=0.88
Fold Results: Precision=0.98, Recall=0.84, F1=0.90
Fold Results: Precision=0.98, Recall=0.82, F1=0.90

Average Metrics from Cross-Validation:
Precision: 0.98
Recall: 0.81
F1 Score: 0.89


In [14]:
#Я пробовал реализовать это и через sklearn. Результаты метрик получил очень схожие.

In [5]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict
from imblearn.over_sampling import SMOTE

# Загрузка данных для обучения
train_data = pd.read_csv('additional_variable_train.csv', delimiter=';', skipinitialspace=True)
print("Columns in train_data:", train_data.columns)

# Подготовка данных для обучения
X = train_data[['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]',
                'Torque [Nm]', 'Tool wear [min]', 'Sum_Parametr', 'Ratio']].values
y = train_data['Machine failure'].values

# Проверка дисбаланса классов
unique, counts = np.unique(y, return_counts=True)
print("Class distribution in training data:", dict(zip(unique, counts)))

# Стандартизация данных
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Балансировка данных с помощью SMOTE
smote = SMOTE(random_state=42)
X_balanced, y_balanced = smote.fit_resample(X, y)

# Проверка распределения классов после балансировки
unique_balanced, counts_balanced = np.unique(y_balanced, return_counts=True)
print("Class distribution in balanced training data:", dict(zip(unique_balanced, counts_balanced)))

# Создаем модель логистической регрессии
model = LogisticRegression(max_iter=1000, random_state=42)

# Применяем кросс-валидацию для предсказания на всех данных
# Используем 5-кратную кросс-валидацию
predictions = cross_val_predict(model, X_balanced, y_balanced, cv=5)

# Оценка модели
accuracy = accuracy_score(y_balanced, predictions)
print("Cross-validated accuracy (Logistic Regression):", accuracy)
print("\nClassification Report (Cross-Validation):\n", classification_report(y_balanced, predictions))
print("\nConfusion Matrix (Cross-Validation):\n", confusion_matrix(y_balanced, predictions))


Columns in train_data: Index(['id', 'Air temperature [K]', 'Process temperature [K]',
       'Rotational speed [rpm]', 'Torque [Nm]', 'Ratio',
       'Deviation from Average', 'Tool wear [min]', 'Machine failure',
       'Tool Wear Failure [TWF]', 'Heat Dissipation Failure [HDF]',
       'Power Failure [PWF]', 'Overstrain Failure [OSF]',
       'Random Failure [RNF]', 'Sum_Parametr', 'Type_H', 'Type_L', 'Type_M'],
      dtype='object')
Class distribution in training data: {np.int64(0): np.int64(134281), np.int64(1): np.int64(2148)}
Class distribution in balanced training data: {np.int64(0): np.int64(134281), np.int64(1): np.int64(134281)}
Cross-validated accuracy (Logistic Regression): 0.8999709564271937

Classification Report (Cross-Validation):
               precision    recall  f1-score   support

           0       0.85      0.96      0.91    134281
           1       0.96      0.84      0.89    134281

    accuracy                           0.90    268562
   macro avg       0.91 